<a href="https://colab.research.google.com/github/s-kamireddy/Logistic-Regression-for-Heart-Attack-Risk-Predictiom/blob/main/projects_in_ai_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import required libraries from titanic example
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import Normalizer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import torch
import warnings
warnings.filterwarnings("ignore")

Part 1:
1. Derive the objective function for Logistic Regression using Maximum Likelihood
Estimation (MLE). Do some research on the MAP technique for Logistic Regression,
include your research on how this technique is different from MLE (include citations).

  >Deriving the logit function:

  Starting with the liklihood function:

  $L(w) = Π_{y_i = 1}(p(x^i)) * Π_{y_i = 1}(1-p(x^i)) $

  $= Π(p(x^i)^{y^i} + (1-p(x^i))^{1-y^i})$

  Our goal is to minimize this function, the minimum of the log will occur at the same place so we can take the logarithm and divide by the number of samples:

  $l(w) =\frac{1}{n} log(Π(p(x^i)^{y^i} + (1-p(x^i))^{1-y^i}))$

  $ = \frac{1}{n}Σ( y^i log(p(x^i)) + (1-y^i)log(1-p(x^i)) )$

  $p(x^i) = \frac{1}{1+ e^{-wx^i}} $

  $l(w) = \frac{1}{n}Σ( y^i log( \frac{1}{1+ e^{-wx^i}}) + (1-y^i)log(1-  \frac{1}{1+ e^{-wx^i}}) )$

  $ = \frac{1}{n}Σ( y^i log( \frac{1}{1+ e^{-wx^i}}) + (1-y^i)log( \frac{e^{-wx^i}}{1+ e^{-wx^i}}) )$

  $= \frac{1}{n}Σ( y^i (log( \frac{1}{1+ e^{-wx^i}}) - log(\frac{e^{-wx^i}}{1+ e^{-wx^i}}) ) + log( \frac{e^{-wx^i}}{1+ e^{-wx^i}}) )$
  $= \frac{1}{n}Σ( y^i (log( \frac{1}{1+ e^{-wx^i}}) - log(\frac{e^{-wx^i}}{1+ e^{-wx^i}}) ) ) +  \frac{1}{n}log( \frac{e^{-wx^i}}{1+ e^{-wx^i}})  $
  $ =  \frac{1}{n}Σ( y^i (log( \frac{1}{1+ e^{-wx^i}}) - log(\frac{1}{1+ e^{-wx^i}}) + log(e^{-wx^i})) ) +  \frac{1}{n}log( \frac{e^{-wx^i}}{1+ e^{-wx^i}}) = \frac{1}{n}Σ( y^i ( log(e^{-wx^i})) ) +  \frac{1}{n}log( \frac{e^{-wx^i}}{1+ e^{-wx^i}})  $
  $=  \frac{1}{n}Σ( -y^iwx^i) +  \frac{1}{n}log( \frac{e^{-wx^i}}{1+ e^{-wx^i}})$

  which is our objective function.

  The MAP or Maximum a Postori estimate is different from MLE, because where MLE only considers the liklihood of the sample data, MAP considers both the liklihood of data as well as a prior belief distribution, with the goal being to find the most likley model parameters given the data, rather than minimize a loss function.

  Source: https://www.cs.cornell.edu/courses/cs4780/2015fa/web/lecturenotes/lecturenote06.html

2. Define a machine learning problem you wish to solve using Logistic Regression. Justify
why logistic regression is the best choice and compare it briefly to another linear
classification model (cite your work if this other technique was not covered in class).

The machine learning problem I would like to solve using logistic regression is the liklihood of a heart attack based on a number of demographic and lifestyle factors. This is because the values are binary (heart attack vs. no heart attack) and we want a liklihood as the output of learning (between 0 and 1) making logistic regression a good choice compared to something like linear regression can output any number in the space of real numbers.

3. Discuss how your dataset corresponds to the variables in your equations, highlighting
any assumptions in your derivation from part 1.

Assuming x is [1, x_1, x_2, ... , x_d] and w is of the form [w_0, w_1, w_2, ..., w_d] (where w_0 is the bias and the rest of the w values correspond to weights). $x^i$ corresponds to the lifestyle factors of each individual




Part 2:
----
1. Link to dataset: https://www.kaggle.com/datasets/iamsouravbanerjee/heart-attack-prediction-dataset


In [ ]:
#2. EDA
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iamsouravbanerjee/heart-attack-prediction-dataset")

print("Path to dataset files:", path)
df = pd.read_csv(path +'/heart_attack_prediction_dataset.csv')

**Data Dictionary**



In [ ]:
df.head()

In [ ]:
df.shape
df.isna().sum()


In [ ]:
df.info()

In [ ]:
df.describe() #dataset stats

In [ ]:
df.describe(include=['O'])

In [ ]:
df["Heart Attack Risk"].value_counts(normalize=True)

In [ ]:

# Convert categorical data to numerical data using cat.codes
df['Sex'] = df['Sex'].astype('category')
df['Sex'] = df['Sex'].cat.codes
df['Exercise Hours Per Week'] = df['Exercise Hours Per Week'].astype('category')
df['Exercise Hours Per Week'] = df['Exercise Hours Per Week'].cat.codes
df['Diet'] = df['Diet'].astype('category')
df['Diet'] = df['Diet'].cat.codes
df['Country'] = df['Country'].astype('category')
df['Country'] = df['Country'].cat.codes
df['Continent'] = df['Continent'].astype('category')
df['Continent'] = df['Continent'].cat.codes
df['Hemisphere'] = df['Hemisphere'].astype('category')
df['Hemisphere'] = df['Hemisphere'].cat.codes

#convert blood pressure to a simple numeric value using Mean Arterial Blood pressure formula
def bp_value(str):
  str = str.split('/')
  return (float(str[0])+ (2*float(str[1])))/3

df['Blood Pressure'] = df['Blood Pressure'].map(bp_value)

#remove patient ID
df.drop('Patient ID', axis=1, inplace=True)



Mean Arterial Blood Pressure formula source: https://clinicalview.gehealthcare.com/white-paper/measuring-mean-arterial-pressure-choosing-most-accurate-method

In [ ]:
df.info()

In [ ]:
#show distributions
df.hist(figsize=(15,15))
plt.show()

In [ ]:
#colinearity
vif_data = pd.DataFrame()
vif_data["feature"] = df.columns
vif_data["VIF"] = [variance_inflation_factor(df.values, i)
                          for i in range(len(df.columns))]
print(vif_data)
#high vif values indicate colinearity

#check which variables are colinear with the covariance matrix
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.show()

We see high rates of correlation between country, hemisphere, and continent (as expected) but also between age, sex, and smoking (presumably because older people and males are more likley to smoke). Therefore we will remove the hemisphere, continent, and smoking features in this dataset.

In [ ]:
df.drop(['Continent', 'Hemisphere', 'Smoking'], axis=1, inplace=True)

In [ ]:
#standardizing the data
for column in df.columns:
    df[column] = (df[column] -
                           df[column].mean()) / df[column].std()

Task 3
----
Now we implement the cost function, as well as the three types of vanilla SGD (batch, minibatch, and stochastic)

In [ ]:
def logit_cost(X, Y, y_hat, w):#cost function
  return -torch.mean(Y * torch.log(y_hat) + (1 - Y) * torch.log(1 - y_hat))



In [ ]:

def gradient_descent_batch( X, Y ):
  losses= []
  w = torch.zeros(X.shape[1], dtype = torch.float64, requires_grad=True)

  for _ in range(10000):
    z = torch.matmul(X, w)
    y_hat = 1 / (1 + torch.exp(-z))
    loss = logit_cost(X, Y, y_hat,w)
    losses.append(loss.item())

    gradient = torch.matmul(X.T, (y_hat - Y)) / X.shape[0]
    w = w - (gradient * .001)

  print(losses)
  plt.plot(losses)
  plt.show()



  return w , losses


In [ ]:
def gradient_descent_minibatch( X, Y ):
  losses= []
  w = torch.zeros(X.shape[1], dtype = torch.float64, requires_grad=True)
  xbatches = []
  ybatches = []
  for i in range(0, 69):
    xbatches.append(X[i:i+127])
    ybatches.append(Y[i:i+127])


  for i in range(10000):
    x_batch = xbatches[i % len(xbatches)]
    y_batch = ybatches[i % len(ybatches)]
    z = torch.matmul(x_batch, w)
    y_hat = 1 / (1 + torch.exp(-z))
    loss = logit_cost(x_batch, y_batch, y_hat, w)
    losses.append(loss.item())

    gradient = torch.matmul(x_batch.T, (y_hat - y_batch)) / x_batch.shape[0]
    w = w - (gradient * .001)

  print(losses)
  plt.plot(losses)
  plt.show()



  return w , losses

In [ ]:
def gradient_descent_stochastic( X, Y ):
  losses= []
  w = torch.zeros(X.shape[1], dtype = torch.float64, requires_grad=True)

  for _ in range(1000):
    x = X[np.random.randint(0, X.shape[0])]
    y = Y[np.random.randint(0, Y.shape[0])]
    z = torch.matmul(x, w)
    y_hat = 1 / (1 + torch.exp(-z))
    loss = logit_cost(x, y, y_hat, w)
    losses.append(loss.item())

    gradient = torch.matmul(x.T, (y_hat - y)) / x.shape[0]
    w = w - (gradient * .01)

  print(losses)
  plt.plot(losses)
  plt.show()



  return w , losses

def test_error(X, Y, W):
  

In [ ]:
def test_error(X, Y, W):
  z = torch.matmul(X, W)
  y_hat = 1 / (1 + torch.exp(-z))

  error = torch.mean(torch.abs(y_hat - Y))
  return error

In [ ]:
Y = df['Heart Attack Risk'].values
X = df.drop('Heart Attack Risk', axis=1).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
x_train = torch.tensor(X_train)
x_train = torch.cat((torch.ones(x_train.shape[0], 1), x_train), dim=1)
y_train = torch.tensor(Y_train, dtype = torch.float64)
x_test = torch.tensor(X_test)
x_test = torch.cat((torch.ones(x_test.shape[0], 1), x_test), dim=1)
y_test = torch.tensor(Y_test, dtype = torch.float64)

In [ ]:
x_train.dtype

In [ ]:
w, loss = gradient_descent_batch(   x_train, y_train)
print(test_error(x_test, y_test, w))


In [ ]:
w, loss = gradient_descent_minibatch(   x_train, y_train)
print(test_error(x_test, y_test, w))



In [ ]:
w, loss = gradient_descent_stochastic(   x_train, y_train)
print(test_error(x_test, y_test, w))


Task 4: Optimization Techniques and Advanced Comparision

In [ ]:
def gradient_descent_stochastic_momentum( X, Y ):
  losses = []
  w = torch.zeros(X.shape[1], dtype = torch.float64, requires_grad=True)
  v_w = torch.zeros(X.shape[1], dtype = torch.float64, requires_grad=True)

  alpha = 0.01
  beta = 0.9
  v = 0

  for _ in range(100000):
    x = X[np.random.randint(0, X.shape[0])]
    y = Y[np.random.randint(0, Y.shape[0])]
    z = torch.matmul(x, w)
    y_hat = logit_cost(x, y, y_hat, w)
    loss = logit_cost(x, y, y_hat, w)
    losses.append(loss.item())

    gradient = torch.matmul(x.T, (y_hat - y)) / x.shape[0]

    v = beta * v + (1 - beta) * gradient
    w = w - (alpha * v)

  print(losses)
  plt.plot(losses)
  plt.show()



  return w , losses



